# Project: 2DEG mobility calculation using mobilitypy¶

The mobility models are implemented using the following refs:

Ref-1: J. Bassaler, J. Mehta, I. Abid, L. Konczewicz, S. Juillaguet, S. Contreras, S. Rennesson, S. Tamariz, M. Nemoz, F. Semond, J. Pernot, F. Medjdoub, Y. Cordier, P. Ferrandis, Al-Rich AlGaN Channel High Electron Mobility Transistors on Silicon: A Relevant Approach for High Temperature Stability of Electron Mobility. Adv. Electron. Mater. 2024, 2400069. https://doi.org/10.1002/aelm.202400069

Ref-2: Zhang, J., Hao, Y., Zhang, J. et al. The mobility of two-dimensional electron gas in AlGaN/GaN heterostructures with varied Al content. Sci. China Ser. F-Inf. Sci. 51, 780–789 (2008). https://doi.org/10.1007/s11432-008-0056-7

Ref-3: 

# 1. Settings

## 1.1 Import modules

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
from pathlib import Path
mobilitypy_tutorial_path = Path().absolute()
sys.path.insert(0, os.path.join(mobilitypy_tutorial_path, '../'))
from mobilitypy import AlloyParams, Mobility2DCarrier, Plottings

In [ ]:
save_figure_dir = os.path.join(mobilitypy_tutorial_path, '../imgs')
savefig = True
fig_dpi = 75

## 1.2 Set physical constants

In [ ]:
alloy_disordered_effect=1
interface_roughness_effect=1
dislocation_effect=1
deformation_potential_effect=1 
piezoelectric_effect=1
acoustic_phonon_effect=1
polar_optical_phonon_effect=1
total_mobility=1
mobility_model='Bassaler'
#==============================
density_2deg = 0.1 # nm^-2
irf_rms_roughness = 0.3 # nm
irf_corr_length = 3.0 # nm
dislocation_density = 1e-4 # nm^-2
occup_dislocation = 0.3
T=300 #K

# 2. Calculate mobilities

In [ ]:
compositions = np.linspace(0,1,101)
mu2deg = Mobility2DCarrier(compositions=compositions, binaries=['AlN', 'GaN'], alloy='AlGaN', system='ternary')

In [ ]:
Temperatures = [10, 100, 200, 300, 400, 500, 600, 700, 800] # in K

In [ ]:
mobility_dff = {}
for T in Temperatures:
    mobility_dff[T] = mu2deg.calculate_sheet_mobility(n_2d=density_2deg, rms_roughness=irf_rms_roughness, 
                                                      corr_len=irf_corr_length, n_dis=dislocation_density, 
                                                      f_dis=occup_dislocation, T=T,
                                                      alloy_disordered_effect=alloy_disordered_effect,
                                                      interface_roughness_effect=interface_roughness_effect,
                                                      dislocation_effect=dislocation_effect,
                                                      deformation_potential_effect=deformation_potential_effect,
                                                      piezoelectric_effect=piezoelectric_effect,
                                                      acoustic_phonon_effect=acoustic_phonon_effect,
                                                      polar_optical_phonon_effect=polar_optical_phonon_effect,
                                                      total_mobility=total_mobility,
                                                      calculate_total_mobility_only=True,
                                                      mobility_model=mobility_model)

In [ ]:
data_FOM_2_plot = {} # Unit of lateral_FOM: MW/cm^2
data_mu_2_plot = {} # Unit of mobility: cm^2/(V.s)
for temperature, mobility_df in mobility_dff.items():
    lateral_FOM = mu2deg.calculate_figure_of_merit(density_2deg, np.array(mobility_df['TOT']), 
                                                   temp=temperature, T_corect_bandgap=False)
    composition_axis = np.array(mobility_df['comp'], dtype=float)
    data_FOM_2_plot[temperature] = np.column_stack((composition_axis, lateral_FOM))# LFOM: MW/cm^2
    data_mu_2_plot[temperature] = np.column_stack((composition_axis, np.array(mobility_df['TOT']))) # mu: cm^2/(V.s)

# 3. Plot mobilities

In [ ]:
plt2deg = Plottings(save_figure_dir=save_figure_dir)
x_label = 'Al composition'
y_label = r'Electron mobility ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)'

In [ ]:
save_file_name_ = f'mobility_T_dependence.png'

fig, ax = plt.subplots(figsize=(9,6),constrained_layout=True)
legend_text = []
for TT, data_2_plot_ in data_mu_2_plot.items():
    fig, ax, _ = plt2deg.plot_2d(data_2_plot_,fig=fig, ax=ax,
                                 ymin=None, ymax=None, xmax=1, xmin=0, y_scale_log=True,
                                 yaxis_label=y_label, xaxis_label=x_label,
                                 show_plot=False)
    legend_text.append(f'{TT}')
ax.legend(legend_text, loc='center left', bbox_to_anchor=(1, 0.5), title='T (K)')
ax.set_ylim(ymin=5e1)
plt2deg.save_figure(save_file_name_,fig=fig, savefig=savefig, dpi=fig_dpi)

In [ ]:
save_file_name_ = f'LFOM_T_dependence.png'

fig, ax = plt.subplots(figsize=(9,6),constrained_layout=True)
legend_text = []
for TT, data_2_plot_ in data_FOM_2_plot.items():
    fig, ax, _ = plt2deg.plot_2d(data_2_plot_,fig=fig, ax=ax,
                                 ymin=None, ymax=None, xmax=1, xmin=0, y_scale_log=True,
                                 yaxis_label=r'LFOM$_\mathrm{AlGaN}$ (MW/$\mathrm{cm}^2$)', 
                                 xaxis_label=x_label, show_plot=False)
    legend_text.append(f'{TT}')
ax.legend(legend_text, loc='center left', bbox_to_anchor=(1, 0.5), title='T (K)')
ax.set_ylim(ymin=4e3)
plt2deg.save_figure(save_file_name_,fig=fig, savefig=savefig, dpi=fig_dpi)

In [ ]:
save_file_name_ = f'LFOM_norm_T_dependence.png'

fig, ax = plt.subplots(figsize=(9,6),constrained_layout=True)
legend_text = []
for TT, data_2_plot in data_FOM_2_plot.items():
    data_2_plot_ = data_2_plot.copy()
    data_2_plot_[:, 1] /= data_2_plot_[0, 1]
    fig, ax, _ = plt2deg.plot_2d(data_2_plot_,fig=fig, ax=ax,
                                 ymin=None, ymax=None, xmax=1, xmin=0, y_scale_log=True,
                                 yaxis_label=r'LFOM$_\mathrm{AlGaN}$ / LFOM$_\mathrm{GaN}$', xaxis_label=x_label,
                                 show_plot=False)
    legend_text.append(f'{TT}')
ax.legend(legend_text, loc='center left', bbox_to_anchor=(1, 0.5), title='T (K)')
ax.axhline(y=1, c='k', ls='--')
plt2deg.save_figure(save_file_name_,fig=fig, savefig=savefig, dpi=fig_dpi)